# Load data and set inventory number as row index

In [189]:
import pandas as pd
import numpy as np
import math
from time import sleep
import requests
import os
working_directory = os.getcwd()
print(working_directory)

birthDateLimit = '1800'
deathDateLimit = '1926'
copyright_applies_date = '1926'

def get_list_from_csv(filename):
    line_list = []
    with open(filename, 'rt', encoding='utf-8') as file_object:
        for one_line in file_object:
            line_list.append(one_line[:-1]) # remove dangling newline from end of line
    return line_list

styles = get_list_from_csv('style_period.csv')

def generate_header_dictionary(accept_media_type):
    user_agent_header = 'VanderBot/1.7.1 (https://github.com/HeardLibrary/linked-data/tree/master/vanderbot; mailto:steve.baskauf@vanderbilt.edu)'
    request_header_dictionary = {
        'Accept' : accept_media_type,
        'Content-Type': 'application/sparql-query',
        'User-Agent': user_agent_header
    }
    return request_header_dictionary

accept_media_type = 'application/json'
requestheader = generate_header_dictionary(accept_media_type)
endpoint = 'https://query.wikidata.org/sparql'

def work_too_old(date_before, work):
    too_old = False
    if not pd.isnull(work['inception_latest_date_val']):
        #print(type(work['inception_latest_date_val'] ))
        #print(work['inception_latest_date_val'] )
        if work['inception_latest_date_val'] < date_before:
            #print('late_date',work['inception_latest_date_val'] )
            too_old = True
    else:
        if not pd.isnull(work['inception_val']):
            if work['inception_val'] < date_before:
                #print('inception_val',work['inception_val'] )
                too_old = True
    return too_old

def old_style_period(styles, work):
    found = False
    if not pd.isnull(work['style_period']):
        for style in styles:
            if style == work['style_period']:
                found = True
    return found

# Query Wikidata Query Service, returns the value of a single property for an item by Q ID
def single_property_values_for_item(pid, qid):
    query = '''
select distinct ?object where {
wd:'''+ qid + ''' wdt:''' + pid + ''' ?object.
}'''
    #print(query)
    r = requests.post(endpoint, data=query.encode('utf-8'), headers=requestheader)
    results_list = []
    try:
    #if 1==1: # replace try: to let errors occur, also comment out the except: clause
        data = r.json()
        #print(data)
        statements = data['results']['bindings']
        if len(statements) > 0: # if no results, the list remains empty
            for statement in statements:
                result_value = statement['object']['value']
                results_list.append(result_value)
    except:
        results_list = [r.text]

    # delay by some amount (quarter second default) to avoid hitting the SPARQL endpoint too rapidly
    sleep(0.1)
    return results_list

def person_too_old(birthDateLimit, qId):
    screen = False
    if not pd.isnull(qId):
        if qId != 'Q4233718': # don't look up unknown artists
            birthDateList = single_property_values_for_item('P569', qId)
            if len(birthDateList) == 0:
                pass
                #print('No birth date given.')
            else:
                birthDate = birthDateList[0][0:10] # all dates are converted to xsd:dateTime and will have a y-m-d date
                #print('This person was born in ', birthDate)
                if birthDate < birthDateLimit:
                    # if the person was born a long time ago, don't retrieve other stuff
                    #print('*** Too old !')
                    screen = True
    return screen

def dead(deathDateLimit, qId):
    screen = False
    if not pd.isnull(qId):
        if qId != 'Q4233718': # don't look up unknown artists
            deathDateList = single_property_values_for_item('P570', qId)
            if len(deathDateList) == 0:
                pass
                #print('No death date given.')
            else:
                deathDate = deathDateList[0][0:10] # all dates are converted to xsd:dateTime and will have a y-m-d date
                #print('This person died in ', deathDate)
                if deathDate < deathDateLimit:
                    # if the person died a long time ago, don't retrieve other stuff
                    screen = True
                    #print('*** Died too long ago !')
    return screen

def work_is_recent(cutoff_date, work):
    is_recent = False
    if not pd.isnull(work['inception_earliest_date_val']):
        if work['inception_earliest_date_val'] >= cutoff_date:
            is_recent = True
    else:
        if not pd.isnull(work['inception_val']):
            if work['inception_val'] >= cutoff_date:
                is_recent = True
    return is_recent


/Users/baskauf/github/vandycite/gallery_works


In [5]:
images_frame = pd.read_csv(working_directory + '/image_list_dedup.csv')
wikidata_frame = pd.read_csv(working_directory + '/works_multiprop.csv') # generates an error saying that the columns have mixed types, but I don't see it
artstor_frame = pd.read_csv(working_directory + '/gallery_works_renamed1.csv')


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (100,101,103,104,106,107,108,118,119,122,124,125,126,128,129,130,133,134) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
wikidata_frame.set_index('inventory_number', inplace=True)
wikidata_frame. head()

,qid,label_en,description_en,inventory_number_uuid,inventory_number_collection,inventory_number_ref1_hash,inventory_number_ref1_statedIn,inventory_number_ref1_referenceUrl,inventory_number_ref1_retrieved_nodeId,inventory_number_ref1_retrieved_val,...,collection_ref1_retrieved_val,collection_ref1_retrieved_prec,location_uuid,location,location_ref1_hash,location_ref1_statedIn,location_ref1_referenceUrl,location_ref1_retrieved_nodeId,location_ref1_retrieved_val,location_ref1_retrieved_prec
inventory_number,,,,,,,,,,,,,,,,,,,,,
2017.001.062,Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,60965535-6F8D-4CB7-B702-D6A84E576DA3,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,87C848D7-1589-4F67-9DBE-12FE22CFF157,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
1994.413,Q102315563,Seated female funerary figure,ceramics by Artist Unknown,9438F518-B646-4E12-AADD-D5FF50DF3346,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,83482B6C-AC70-46B8-AC76-2E65F1C42351,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
1969.014,Q102315787,Head of a Young Girl or a Goddess,sculpture by Artist Unknown,B5F33FEE-A2B6-4D77-8764-C3942DC09E1B,Q18563658,09cfc74e2a883ae6ee704d256bb7fc57733abd97,NaN,https://library.artstor.org/#/asset/26753561,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,320CD258-94F0-420A-B386-EA2EEB98A3B6,Q18563658,09cfc74e2a883ae6ee704d256bb7fc57733abd97,NaN,https://library.artstor.org/#/asset/26753561,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
2013.042,Q102949359,"'I Hate You For Hitting My Mother,’ Minneapolis",photograph by Donna Ferrato,AE86C1A8-48F0-4FD4-92D2-6DB10C878E24,Q18563658,c4d5f5b1f6a4c2840c84ddcb9500194882522c81,NaN,https://library.artstor.org/#/asset/26759879,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,0F670FF6-44C7-4243-B8E8-ECC5F493EF66,Q18563658,c4d5f5b1f6a4c2840c84ddcb9500194882522c81,NaN,https://library.artstor.org/#/asset/26759879,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
1992.139 a-c,Q102958387,(An Imaginary) Night Attack in the War Between...,print by Yoshitora,B7A0A58E-8E8B-4B6A-A1E0-1EE854BA62B0,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,1F5DA01A-7D81-492D-8758-DEE5FD1226E4,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0


In [7]:
images_frame.set_index('accession', inplace=True)
images_frame.head()

,has_image
accession,
1956.001,yes
1956.002,yes
1956.003,yes
1956.004,yes
1956.005,yes


In [8]:
artstor_frame.set_index('accession_number', inplace=True)
artstor_frame.head()

,ssid,filename,title,creator_string,date,classification,medium,measurements,style_period,country_culture,...,date_acquired,gift_of,purchased_from,credit_line,provenance,collection,last_change,notes,rights,media_url
accession_number,,,,,,,,,,,,,,,,,,,,,
2020.005,28884744,2020.005.tif,G2 Cover 11.06.2020,"Himid, Lubaina",2020,Prints,print on Hahnemühle Photorag Fine Art paper,12 x 10 in.,NaN,United Kingdom,...,2020,NaN,The Guardian Print Shop (Gallery Purchase),Collection of Vanderbilt University Fine Arts ...,NaN,NaN,NaN,NaN,IN COPYRIGHT,https://forum.jstor.org/assets/28884744/repres...
2020.003,28884738,NaN,Portrait of Julián Trujillo Largacha,"Mendoza, Pantaléon",late 19th century CE,Paintings,oil on canvas,NaN,NaN,Colombia,...,NaN,"Lisa Carr in honor of her father, Albert Trujillo",NaN,Collection of Vanderbilt University Fine Arts ...,Descended through donor's father's family.,NaN,NaN,NaN,NO KNOWN COPYRIGHT,NaN
2020.004,28884737,2020.004.jpg,Manhattan Skyline,"Gallagher, Sears (American painter, illustrato...",NaN,NaN,etching,12.25 x 9 in.,NaN,United States,...,2020,NaN,Wiscasset Bay Gallery (Fine Arts Gallery Purch...,NaN,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/28884737/repres...
2020.002,28589522,2020-002.jpg,Relics,"Woodruff, Hale Aspacio (American painter, 1900...",1935 CE,Graphic Arts,linocuts (prints),9 x 11.25 in,NaN,United States,...,2020,NaN,Purchased with The Joseph L. May History of Pr...,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,NaN,NaN,COPYRIGHT UNDETERMINED,https://forum.jstor.org/assets/28589522/repres...
2020.001,28262628,2020-001.tif,"The Chimneys, Portsmouth","Hassam, Childe (American painter and printmake...",1915 CE,Printmaking,etching,NaN,NaN,United States,...,2020,Purchased with Research Funds from the Andrew ...,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,NaN,NaN,NO KNOWN COPYRIGHT,https://forum.jstor.org/assets/28262628/repres...


# Remove duplicate column headers and join tables

Since all Artstor items are in Wikidata, an outer join should match everything.

In [9]:
# get rid of duplicate column names
wikidata_frame.drop('collection', axis=1, inplace=True)
wikidata_frame.drop('title', axis=1, inplace=True)

result = pd.concat([wikidata_frame, artstor_frame], axis=1, join="outer")
result.head()

,qid,label_en,description_en,inventory_number_uuid,inventory_number_collection,inventory_number_ref1_hash,inventory_number_ref1_statedIn,inventory_number_ref1_referenceUrl,inventory_number_ref1_retrieved_nodeId,inventory_number_ref1_retrieved_val,...,date_acquired,gift_of,purchased_from,credit_line,provenance,collection,last_change,notes,rights,media_url
2017.001.062,Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,60965535-6F8D-4CB7-B702-D6A84E576DA3,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2017,"Walter F. Schatz and Mary Pullig Schatz, MD",NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Mon Nov 26 00:00:00 EST 2018,Gift acquisition paperwork in file for 2017.00...,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/26760365/repres...
1994.413,Q102315563,Seated female funerary figure,ceramics by Artist Unknown,9438F518-B646-4E12-AADD-D5FF50DF3346,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,NaN,Marjorie and Leon Marlowe,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Marjorie and Leon Marlowe Collection,Mon Nov 12 00:00:00 EST 2018,#77\nDr. Headrick has three theories about the...,NaN,https://forum.jstor.org/assets/26757434/repres...
1969.014,Q102315787,Head of a Young Girl or a Goddess,sculpture by Artist Unknown,B5F33FEE-A2B6-4D77-8764-C3942DC09E1B,Q18563658,09cfc74e2a883ae6ee704d256bb7fc57733abd97,NaN,https://library.artstor.org/#/asset/26753561,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,1969,Mr. and Mrs. David K. Wilson,Mathias Komor,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Fri Oct 26 00:00:00 EDT 2018,Note: Broken off at neck (from a full figure)....,NO KNOWN COPYRIGHT,https://forum.jstor.org/assets/26753561/repres...
2013.042,Q102949359,"'I Hate You For Hitting My Mother,’ Minneapolis",photograph by Donna Ferrato,AE86C1A8-48F0-4FD4-92D2-6DB10C878E24,Q18563658,c4d5f5b1f6a4c2840c84ddcb9500194882522c81,NaN,https://library.artstor.org/#/asset/26759879,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2013,"Scott and Missy Tannen (both bachelor of arts,...",NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Mon Nov 26 00:00:00 EST 2018,"Image; sheet: 24"" x 20""\n\nThe acquisition of ...",IN COPYRIGHT,https://forum.jstor.org/assets/26759879/repres...
1992.139 a-c,Q102958387,(An Imaginary) Night Attack in the War Between...,print by Yoshitora,B7A0A58E-8E8B-4B6A-A1E0-1EE854BA62B0,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,1992,Herman D. Doochin,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Herman D. Doochin Collection,Wed Oct 03 00:00:00 EDT 2018,"A: Left panel; H. 14-1/4"" W.9-3/4"" ; artist's...",NaN,https://forum.jstor.org/assets/26756454/repres...


Do an inner join since we don't care about images not associated with items, nor items without images.

Basically, this just deletes works that don't have images from the earlier join.

In [181]:
result = pd.concat([images_frame, result], axis=1, join="inner")
# Don't need the has_image column any more
result.drop('has_image', axis=1, inplace=True)
works = result.sort_values(by=['qid']).copy()
works.head()

,qid,label_en,description_en,inventory_number_uuid,inventory_number_collection,inventory_number_ref1_hash,inventory_number_ref1_statedIn,inventory_number_ref1_referenceUrl,inventory_number_ref1_retrieved_nodeId,inventory_number_ref1_retrieved_val,...,date_acquired,gift_of,purchased_from,credit_line,provenance,collection,last_change,notes,rights,media_url
2017.001.062,Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,60965535-6F8D-4CB7-B702-D6A84E576DA3,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2017,"Walter F. Schatz and Mary Pullig Schatz, MD",NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Mon Nov 26 00:00:00 EST 2018,Gift acquisition paperwork in file for 2017.00...,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/26760365/repres...
1994.413,Q102315563,Seated female funerary figure,ceramics by Artist Unknown,9438F518-B646-4E12-AADD-D5FF50DF3346,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,NaN,Marjorie and Leon Marlowe,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Marjorie and Leon Marlowe Collection,Mon Nov 12 00:00:00 EST 2018,#77\nDr. Headrick has three theories about the...,NaN,https://forum.jstor.org/assets/26757434/repres...
1992.139 a-c,Q102958387,(An Imaginary) Night Attack in the War Between...,print by Yoshitora,B7A0A58E-8E8B-4B6A-A1E0-1EE854BA62B0,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,1992,Herman D. Doochin,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Herman D. Doochin Collection,Wed Oct 03 00:00:00 EDT 2018,"A: Left panel; H. 14-1/4"" W.9-3/4"" ; artist's...",NaN,https://forum.jstor.org/assets/26756454/repres...
2016.002.01,Q102961140,#109,print by Bruce Conner,CCA2E88D-EB3F-4083-993E-A694035A3C8A,Q18563658,de2a6faedad59c4c8077dd14390161d2b190af84,NaN,https://library.artstor.org/#/asset/26760161,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2016,Conner Family Trust,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Tue Nov 29 00:00:00 EST 2016,From a series of offset lithographs made from ...,IN COPYRIGHT,https://forum.jstor.org/assets/26760161/repres...
2016.002.03,Q102961146,#116,print by Bruce Conner,A762EC6B-9E4C-49FE-94B1-47562347D122,Q18563658,dd49fd11eced4e9aecf9b06a7d02fdf6c4ea71bc,NaN,https://library.artstor.org/#/asset/26760163,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2016,Conner Family Trust,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Tue Nov 29 00:00:00 EST 2016,From a series of offset lithographs made from ...,IN COPYRIGHT,https://forum.jstor.org/assets/26760163/repres...


In [163]:
works.to_csv(working_directory + '/items_with_images_full_metadata.csv')

In [182]:
# Add an empty status column to be filled in
works['status'] = ''
headers = list(works.columns)
headers.sort()
headers

['classification',
 'collection',
 'collection_end_time_nodeId',
 'collection_end_time_prec',
 'collection_end_time_val',
 'collection_ref1_hash',
 'collection_ref1_referenceUrl',
 'collection_ref1_retrieved_nodeId',
 'collection_ref1_retrieved_prec',
 'collection_ref1_retrieved_val',
 'collection_ref1_statedIn',
 'collection_start_time_nodeId',
 'collection_start_time_prec',
 'collection_start_time_val',
 'collection_uuid',
 'copyright_status',
 'copyright_status_applies_to_jurisdiction',
 'copyright_status_determination_method',
 'copyright_status_ref1_hash',
 'copyright_status_ref1_referenceUrl',
 'copyright_status_ref1_retrieved_nodeId',
 'copyright_status_ref1_retrieved_prec',
 'copyright_status_ref1_retrieved_val',
 'copyright_status_ref1_statedIn',
 'copyright_status_uuid',
 'country',
 'country_culture',
 'country_of_origin',
 'country_of_origin_ref1_hash',
 'country_of_origin_ref1_referenceUrl',
 'country_of_origin_ref1_retrieved_nodeId',
 'country_of_origin_ref1_retrieved_pre

In [193]:
#row = 173
#print(works.iloc[row])
#print(works.iloc[row]['inception_latest_date_val'])
#for nothing in range(1):
count = 0
for accession, work in works.iterrows():
    status = 'unknown'
    count += 1
    
    if work_too_old(copyright_applies_date, work):
        status = 'inception prior to copyright cutoff'
    elif old_style_period(styles, work):
        status = 'from style or period that ended prior to copyright cutoff'
    elif dead(deathDateLimit, work['creator']):
        status = 'artist died before copyright cutoff'
    elif person_too_old(birthDateLimit, work['creator']):
        status = 'artist was born before '+ birthDateLimit
    elif not pd.isnull(work['rights']):
        rights = work['rights']
        #print(rights)
        if rights == 'NO KNOWN COPYRIGHT' or rights == 'NO COPYRIGHT – UNITED STATES':
            status = 'assessed to be out of copyright'
        elif rights == 'IN COPYRIGHT':
            status = 'assessed to in copyright'

    # If none of the status checks came up with anything, do a last check on the date
    if status == 'unknown':
        if work_is_recent(copyright_applies_date, work):
            status = 'inception was after copyright cutoff'

    works.at[accession,'status'] = status
    print(count, accession, status)

print('done')

1 2017.001.062 inception was after copyright cutoff
2 1994.413 inception prior to copyright cutoff
3 1992.139 a-c inception prior to copyright cutoff
4 2016.002.01 assessed to in copyright
5 2016.002.03 assessed to in copyright
6 2016.002.04 assessed to in copyright
7 1991.007 inception prior to copyright cutoff
8 2004.065 inception was after copyright cutoff
9 1984.021 inception prior to copyright cutoff
10 2004.029 artist died before copyright cutoff
11 1992.141 inception prior to copyright cutoff
12 1992.083 inception prior to copyright cutoff
13 1956.001 inception prior to copyright cutoff
14 1969.023 inception prior to copyright cutoff
15 1969.025 inception prior to copyright cutoff
16 1979.0912P artist died before copyright cutoff
17 2016.011.19 assessed to in copyright
18 2004.030 artist died before copyright cutoff
19 1956.051 inception prior to copyright cutoff
20 1972.053 inception prior to copyright cutoff
21 1988.097 inception prior to copyright cutoff
22 1975.054 inception

201 1979.0379P unknown
202 1979.0691P inception was after copyright cutoff
203 1979.0085 assessed to be out of copyright
204 1992.341 assessed to be out of copyright
205 1988.140 assessed to be out of copyright
206 1992.232 assessed to be out of copyright
207 1994.186 from style or period that ended prior to copyright cutoff
208 1994.220 inception prior to copyright cutoff
209 2004.001 inception was after copyright cutoff
210 1986.022 unknown
211 1990.006 unknown
212 1993.192 unknown
213 1988.022 inception was after copyright cutoff
214 1980.112 inception was after copyright cutoff
215 1988.024 inception was after copyright cutoff
216 1988.021 unknown
217 1992.082 inception prior to copyright cutoff
218 1975.006 inception prior to copyright cutoff
219 1999.172z inception prior to copyright cutoff
220 1999.172c inception prior to copyright cutoff
221 1994.126 unknown
222 1972.052 inception prior to copyright cutoff
223 1999.242 from style or period that ended prior to copyright cutoff
2

403 2016.011.07 assessed to in copyright
404 2015.075.45 unknown
405 2016.011.02 assessed to in copyright
406 2015.038cc assessed to in copyright
407 1982.084 inception was after copyright cutoff
408 1980.013 artist died before copyright cutoff
409 1965.020 inception prior to copyright cutoff
410 2015.075.47 unknown
411 2015.007 inception was after copyright cutoff
412 2015.008 inception was after copyright cutoff
413 1979.0643P inception prior to copyright cutoff
414 1956.015 inception prior to copyright cutoff
415 2016.011.04 assessed to in copyright
416 1992.503A&B assessed to be out of copyright
417 1987.037 assessed to be out of copyright
418 1993.061 unknown
419 1988.046 inception prior to copyright cutoff
420 2003.051 inception was after copyright cutoff
421 1988.012 unknown
422 1994.408 inception prior to copyright cutoff
423 1994.376 inception prior to copyright cutoff
424 1970.030 inception prior to copyright cutoff
425 1989.024 artist died before copyright cutoff
426 2007.04

593 1997.157 inception was after copyright cutoff
594 1989.044 inception was after copyright cutoff
595 1994.192 unknown
596 1999.172w inception prior to copyright cutoff
597 1988.130 assessed to be out of copyright
598 1995.068 unknown
599 1995.067 from style or period that ended prior to copyright cutoff
600 1988.131 assessed to be out of copyright
601 2015.021 assessed to in copyright
602 2004.010 inception was after copyright cutoff
603 2001.045 inception was after copyright cutoff
604 2013.038 assessed to in copyright
605 1978.038 inception prior to copyright cutoff
606 1994.190 inception prior to copyright cutoff
607 1996.171 inception prior to copyright cutoff
608 2000.112 inception prior to copyright cutoff
609 1993.175 unknown
610 1999.172p inception prior to copyright cutoff
611 2007.070b inception was after copyright cutoff
612 1979.0887P inception prior to copyright cutoff
613 1988.039 inception was after copyright cutoff
614 1970.048 inception prior to copyright cutoff
615

789 1999.173 inception was after copyright cutoff
790 1965.003 assessed to in copyright
791 1994.543 inception was after copyright cutoff
792 1999.172x inception prior to copyright cutoff
793 1999.172b inception prior to copyright cutoff
794 1992.084 inception prior to copyright cutoff
795 2010.005 inception prior to copyright cutoff
796 1979.0215P inception prior to copyright cutoff
797 1965.038 inception prior to copyright cutoff
798 2007.071b inception was after copyright cutoff
799 1979.0915P artist died before copyright cutoff
800 2004.039 inception prior to copyright cutoff
801 1979.0754P unknown
802 1979.0753P unknown
803 1989.045 inception was after copyright cutoff
804 1994.332 from style or period that ended prior to copyright cutoff
805 2016.011.03 assessed to in copyright
806 2003.050 inception prior to copyright cutoff
807 2007.051 artist died before copyright cutoff
808 2013.006 assessed to in copyright
809 2017.001.053a inception was after copyright cutoff
810 2017.001.0

...

4850 1982.090 inception was after copyright cutoff
4851 1982.098 inception was after copyright cutoff
4852 1968.006 inception prior to copyright cutoff
4853 1983.005 inception was after copyright cutoff
4854 1983.008 unknown
4855 1965.031 inception was after copyright cutoff
4856 1983.009 unknown
4857 1957.046 inception was after copyright cutoff
4858 1957.035 inception was after copyright cutoff
4859 1992.391A-F unknown
4860 1984.045 unknown
4861 1992.378 unknown
4862 1992.293 unknown
4863 1992.290 unknown
4864 1992.292 unknown
4865 1985.020 unknown
4866 1992.291 unknown
4867 1985.044 assessed to be out of copyright
4868 1992.289 unknown
4869 1992.288 unknown
4870 1992.287 unknown
4871 1992.286 unknown
4872 1987.002 unknown
4873 1992.284 unknown
4874 1992.285 unknown
4875 1988.008 unknown
4876 1992.283 unknown
4877 1992.282 unknown
4878 1992.281 unknown
4879 1992.280 unknown
4880 1989.032 inception prior to copyright cutoff
4881 1992.279 unknown
4882 1992.278 unknown
4883 1989.043 inc

5048 2009.012g assessed to in copyright
5049 2009.012a assessed to in copyright
5050 2009.013a assessed to in copyright
5051 2000.077 inception was after copyright cutoff
5052 1999.163 inception was after copyright cutoff
5053 2009.007i assessed to in copyright
5054 2012.002 inception prior to copyright cutoff
5055 2009.004r assessed to in copyright
5056 1973.020 inception prior to copyright cutoff
5057 1979.0924p inception prior to copyright cutoff
5058 2015.075.32 unknown
5059 2009.011l assessed to in copyright
5060 2009.011p assessed to in copyright
5061 2009.007k assessed to in copyright
5062 2009.012j assessed to in copyright
5063 2009.009n assessed to in copyright
5064 2009.003a assessed to in copyright
5065 2009.008i assessed to in copyright
5066 1979.1183P inception prior to copyright cutoff
5067 2005.001e inception was after copyright cutoff
5068 2009.003p assessed to in copyright
5069 1994.155 unknown
5070 1994.086 unknown
5071 1992.097 inception prior to copyright cutoff
507

5250 2003.069 inception was after copyright cutoff
5251 2003.068 inception was after copyright cutoff
5252 2004.041 inception was after copyright cutoff
5253 1992.319 artist died before copyright cutoff
5254 1992.328 artist died before copyright cutoff
5255 1992.320 artist died before copyright cutoff
5256 1992.327 artist died before copyright cutoff
5257 1998.062 assessed to be out of copyright
5258 2003.067 inception was after copyright cutoff
5259 1979.1022P assessed to be out of copyright
5260 2004.081 inception was after copyright cutoff
5261 2004.042 inception was after copyright cutoff
5262 1979.0911P assessed to be out of copyright
5263 2003.062 inception was after copyright cutoff
5264 2015.006 inception was after copyright cutoff
5265 1979.0469P unknown
5266 1999.059 inception was after copyright cutoff
5267 1979.0198P unknown
5268 1997.108a inception was after copyright cutoff
5269 1997.108b inception was after copyright cutoff
5270 1997.108c inception was after copyright cu

5422 1997.036 inception was after copyright cutoff
5423 1997.037 inception was after copyright cutoff
5424 1997.039 inception was after copyright cutoff
5425 1972.024 inception prior to copyright cutoff
5426 1978.073 inception prior to copyright cutoff
5427 1988.028 unknown
5428 1969.016 inception prior to copyright cutoff
5429 1992.251 unknown
5430 1992.331 inception was after copyright cutoff
5431 1978.103a inception prior to copyright cutoff
5432 1957.041 inception prior to copyright cutoff
5433 1997.043 inception was after copyright cutoff
5434 1978.082 inception prior to copyright cutoff
5435 1997.044 unknown
5436 1978.099 assessed to be out of copyright
5437 1978.064 inception prior to copyright cutoff
5438 1992.258 unknown
5439 1978.027 artist died before copyright cutoff
5440 2014.049 inception was after copyright cutoff
5441 1956.012 inception prior to copyright cutoff
5442 1969.018 inception was after copyright cutoff
5443 1956.022 inception prior to copyright cutoff
5444 197

5623 1983.011 inception was after copyright cutoff
5624 1992.062 unknown
5625 2011.002 unknown
5626 2011.003 unknown
5627 1992.337a artist died before copyright cutoff
5628 1992.337b artist died before copyright cutoff
5629 1979.0673P inception was after copyright cutoff
5630 1997.152 inception prior to copyright cutoff
5631 1999.227 from style or period that ended prior to copyright cutoff
5632 1992.177 unknown
5633 1986.064 assessed to be out of copyright
5634 1986.055 from style or period that ended prior to copyright cutoff
5635 1979.0449P assessed to be out of copyright
5636 1979.0450P assessed to be out of copyright
5637 1999.249 inception was after copyright cutoff
5638 1992.057 assessed to be out of copyright
5639 1992.058 assessed to be out of copyright
5640 1992.060 assessed to be out of copyright
5641 1992.055 assessed to be out of copyright
5642 1992.056 assessed to be out of copyright
5643 1992.059 unknown
5644 1980.034 unknown
5645 1976.036 assessed to be out of copyright

In [196]:
output_df = works.filter(['qid', 'label_en', 'description_en', 'creator', 'instance_of', 'inception_val', 'inception_earliest_date_val', 'inception_latest_date_val', 'height_val', 'width_val', 'thickness_val', 'diameter_val', 'style_period', 'rights', 'media_url', 'status'], axis = 1)
output_df.head()

,qid,label_en,description_en,creator,instance_of,inception_val,inception_earliest_date_val,inception_latest_date_val,height_val,width_val,thickness_val,diameter_val,style_period,rights,media_url,status
2017.001.062,Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,Q2605345,Q11060274,1951-01-01T00:00:00Z,1950-01-01T00:00:00Z,1952-01-01T00:00:00Z,13.00,17.00,NaN,NaN,NaN,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/26760365/repres...,inception was after copyright cutoff
1994.413,Q102315563,Seated female funerary figure,ceramics by Artist Unknown,Q4233718,Q13464614,-0299-01-01T00:00:00Z,-0799-01-01T00:00:00Z,0200-01-01T00:00:00Z,7.75,5.75,2.0,NaN,Pre-Columbian (American),NaN,https://forum.jstor.org/assets/26757434/repres...,inception prior to copyright cutoff
1992.139 a-c,Q102958387,(An Imaginary) Night Attack in the War Between...,print by Yoshitora,Q3075041,Q11060274,1850-01-01T00:00:00Z,1847-01-01T00:00:00Z,1853-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,https://forum.jstor.org/assets/26756454/repres...,inception prior to copyright cutoff
2016.002.01,Q102961140,#109,print by Bruce Conner,Q991589,Q11060274,1970-01-01T00:00:00Z,NaN,NaN,5.25,5.50,NaN,NaN,Abstract,IN COPYRIGHT,https://forum.jstor.org/assets/26760161/repres...,assessed to in copyright
2016.002.03,Q102961146,#116,print by Bruce Conner,Q991589,Q11060274,1970-01-01T00:00:00Z,NaN,NaN,7.75,10.75,NaN,NaN,Abstract,IN COPYRIGHT,https://forum.jstor.org/assets/26760163/repres...,assessed to in copyright


In [197]:
works.to_csv(working_directory + '/items_with_status.csv')
output_df.to_csv(working_directory + '/items_status_abbrev.csv')

In [199]:
summary = output_df.groupby(['status'], sort=False).size().reset_index(name='Count')
summary

,status,Count
0,inception was after copyright cutoff,1660
1,inception prior to copyright cutoff,1119
2,assessed to in copyright,988
3,artist died before copyright cutoff,192
4,assessed to be out of copyright,437
5,unknown,934
6,from style or period that ended prior to copyr...,352
7,artist was born before 1800,2
